In [5]:
#importing libraries


import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv("preprocessed_data")
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Total_Area,Total_Bathrooms
0,5,RL,65.0,8450,1,1,3,Lvl,Inside,0,...,3,4,None,0,1,2,WD,Normal,2566.0,3.5
1,0,RL,80.0,9600,1,1,3,Lvl,FR2,0,...,3,4,None,0,4,1,WD,Normal,2524.0,2.5
2,5,RL,68.0,11250,1,1,0,Lvl,Inside,0,...,3,4,None,0,8,2,WD,Normal,2706.0,3.5
3,6,RL,60.0,9550,1,1,0,Lvl,Corner,0,...,3,4,None,0,1,0,WD,Abnorml,2473.0,2.0
4,5,RL,84.0,14260,1,1,0,Lvl,FR2,0,...,3,4,None,0,11,2,WD,Normal,3343.0,3.5


In [7]:
#importing datasets


trainset = pd.read_csv("train.csv")
testset = pd.read_csv("test.csv")

train_id = trainset.Id
test_id = testset.Id
train_salesprice = trainset.SalePrice

In [8]:
df.drop(['TotalBsmtSF', '1stFlrSF', '2ndFlrSF'], axis=1, inplace=True)
df.drop(['BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath'], axis=1, inplace=True)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Total_Area,Total_Bathrooms
0,5,RL,65.0,8450,1,1,3,Lvl,Inside,0,...,3,4,None,0,1,2,WD,Normal,2566.0,3.5
1,0,RL,80.0,9600,1,1,3,Lvl,FR2,0,...,3,4,None,0,4,1,WD,Normal,2524.0,2.5
2,5,RL,68.0,11250,1,1,0,Lvl,Inside,0,...,3,4,None,0,8,2,WD,Normal,2706.0,3.5
3,6,RL,60.0,9550,1,1,0,Lvl,Corner,0,...,3,4,None,0,1,0,WD,Abnorml,2473.0,2.0
4,5,RL,84.0,14260,1,1,0,Lvl,FR2,0,...,3,4,None,0,11,2,WD,Normal,3343.0,3.5


In [9]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Total_Area,Total_Bathrooms
0,5,RL,65.0,8450,1,1,3,Lvl,Inside,0,...,3,4,None,0,1,2,WD,Normal,2566.0,3.5
1,0,RL,80.0,9600,1,1,3,Lvl,FR2,0,...,3,4,None,0,4,1,WD,Normal,2524.0,2.5
2,5,RL,68.0,11250,1,1,0,Lvl,Inside,0,...,3,4,None,0,8,2,WD,Normal,2706.0,3.5
3,6,RL,60.0,9550,1,1,0,Lvl,Corner,0,...,3,4,None,0,1,0,WD,Abnorml,2473.0,2.0
4,5,RL,84.0,14260,1,1,0,Lvl,FR2,0,...,3,4,None,0,11,2,WD,Normal,3343.0,3.5


In [11]:
#Getting the dummy variables for our categorical columns

df = pd.get_dummies(df)
print(df.shape)

(2919, 215)


In [13]:
x = df[:1460]
test = df[1460:]
y = train_salesprice

In [14]:
import xgboost as xgb

from sklearn.model_selection import train_test_split 

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)


In [15]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

dep = np.arange(1,9)
weight = np.arange(1,9)

param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [5, 6, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200],
        'objective': ['reg:squarederror']
}

#XGBRegressor classifier
classifiers = XGBRegressor()

clf_cv = GridSearchCV(classifiers, param_grid=param_tuning,cv=6)

clf_cv.fit(x,y)

# Print the tuned parameter and score
print("Tuned XGBoost Parameters: {}".format(clf_cv.best_params_))
print("Best score is {}".format(clf_cv.best_score_))

Tuned XGBoost Parameters: {'colsample_bytree': 0.5, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 3, 'n_estimators': 200, 'objective': 'reg:squarederror', 'subsample': 0.7}
Best score is 0.8826781016008348


In [16]:
Y_pred = clf_cv.predict(test)

In [17]:
testset['SalePrice'] = Y_pred
testset['Id'] = test_id
testset[['Id', 'SalePrice']].to_csv('xgb_submission2.csv', index=False)